# テストデータ作成用Notebook
## Summary
- 3次元ビンパッキングの精度を確認するためのテストデータを作成するNotebook
- 以下の条件で、ランダムな組合せを20パターン作成する
  - 1パターンあたりの荷物の個数は200個
  - サイズのバリエーションは以下の10種類（長さの単位はmm、重さの単位はkg）

    |size_name|width|height|depth|weight|notes|
    |---|---|---|---|---|---|
    |b1|270|130|200|1|クロネコボックス6|
    |b2|325|140|235|1||
    |b3|380|150|270|1|クロネコボックス8|
    |b4|380|250|270|1||
    |b5|380|350|270|1|クロネコボックス10|
    |b6|455|320|325|1||
    |b7|530|290|380|1|クロネコボックス12|
    |b8|565|345|390|1||
    |b9|600|400|400|1|クロネコボックス14|
    |b10|645|455|410|1||


## config

In [1]:
# 作成するパターン数
n_pattern = 20
# 1パターンあたりのボックス総数
n_total_box = 200

## main


In [2]:
from pathlib import Path
from random import randint
import pandas as pd

In [3]:
def decompose(n_total: int, n_size: int) -> pd.Series:
    total = 0
    size_count = {}

    for i in range(n_size-1):
        if total == n_total: c = 0
        else:
            c = randint(0,40)
            if total + c > 200: c = 200 - total
        size_count[i] = c
        total += c
    size_count[n_size] = 200 - total

    return pd.Series(size_count).reset_index(drop=True)

In [4]:
path_root = Path.cwd().parent

In [5]:
# ボックスサイズ一覧を読み込み
box_size = pd.read_csv(path_root/"config/box_size.csv")
box_size

,size_name,width,height,depth,weight
0,b1,270,130,200,1
1,b2,325,140,235,1
2,b3,380,150,270,1
3,b4,380,250,270,1
4,b5,380,350,270,1
5,b6,455,320,325,1
6,b7,530,290,380,1
7,b8,565,345,390,1
8,b9,600,400,400,1
9,b10,645,455,410,1


In [6]:
# ランダムに個数を振り分ける
for i in range(1, n_pattern+1):
    box_size[f"pattern_{i:02d}"] = decompose(n_total_box, box_size.shape[0])
    assert box_size[f"pattern_{i:02d}"].sum() == 200

In [7]:
box_size

,size_name,width,height,depth,weight,pattern_01,pattern_02,pattern_03,pattern_04,pattern_05,...,pattern_11,pattern_12,pattern_13,pattern_14,pattern_15,pattern_16,pattern_17,pattern_18,pattern_19,pattern_20
0,b1,270,130,200,1,35,25,13,33,32,...,25,18,19,39,8,26,11,29,19,22
1,b2,325,140,235,1,33,30,16,26,32,...,17,2,5,18,21,40,7,39,12,22
2,b3,380,150,270,1,17,13,21,4,28,...,1,2,37,40,21,7,13,7,33,30
3,b4,380,250,270,1,6,38,40,26,2,...,18,39,24,0,4,39,9,13,8,10
4,b5,380,350,270,1,38,25,12,3,28,...,0,34,17,34,0,36,37,0,10,24
5,b6,455,320,325,1,36,6,9,25,10,...,22,6,28,13,18,36,3,31,27,39
6,b7,530,290,380,1,10,8,15,38,34,...,18,29,3,26,30,14,30,37,19,15
7,b8,565,345,390,1,17,12,36,37,23,...,7,11,21,30,9,2,9,16,3,5
8,b9,600,400,400,1,8,19,32,8,11,...,22,34,1,0,21,0,18,26,4,27
9,b10,645,455,410,1,0,24,6,0,0,...,70,25,45,0,68,0,63,2,65,6


In [8]:
# 出力
box_size.to_csv(path_root/"data/test_data.csv", index=False)